# 
# Utilização de Algorítmos de Machine Learning para Identificação de Empresas "de Fachada" em Operações de Importação
TCC PUC Minas - LUCIANA MACEDO RODRIGUES

### NOTEBOOK 4 - MACHINE LEARNING - CLASSIFICAÇÃO

### 1 - Configurações iniciais

In [ ]:
# Instalando o Pycaret

In [ ]:
pip install  pycaret

In [ ]:
# Importando os pacotes necessários
import pandas as pd
import numpy as np
from pycaret.classification import *
from pycaret.utils import check_metric

### 2 - Leitura do dataframe final

In [ ]:
df_final = pd.read_csv('arquivos/df_final.csv', sep = ',')
df_final.head()

In [ ]:
# Dimensões do dataframe final
df_final.shape

In [ ]:
# Resumo de informações do dataframe final 
df_final.info()

In [ ]:
# Corrigindo o tipo da variável alvo
df_final = df_final.astype({'TARGET': 'object'})

### 3 - Isolamento de uma amostra para predições simulando um cenário real

In [ ]:
# Separando 5% da amostra aleatóriamente
data = df_final.sample(frac=0.95, random_state=786)
data_pred = df_final.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_pred.reset_index(inplace=True, drop=True)
print('Dados para Modelagem (data) correspondendo a 95% do df_final: ' + str(data.shape))
print('Dados para Predições (data_pred) correspondendo a 5% do df_final: ' + str(data_pred.shape))

Como não há estratificação dessa amostra, assim como no mundo real, ela poderá não ter nenhum positivo.

### 4 - Configurando o ambiente no Pycaret

Antes de rodar os algorítmos de classificação no Pycaret é preciso chamar a função 'setup' que inicializa o ambiente de treinamento e cria o 'pipeline' de transformação.

Para referência aos testes realizados para a seleção de parâmetros de setup consultar o NOTEBOOK05.

In [ ]:
classif_exp = setup(data = data,
                   target = 'TARGET',
                   session_id=123,
                   silent = True,
                   ordinal_features = {'EMP_PORTE':['ME','EPP','DEMAIS']}, 
                   fix_imbalance = True,
                   high_cardinality_features = ['UF_EMP'],
                   ignore_low_variance = True,
                   feature_ratio = True,
                   remove_multicollinearity = True, 
                   multicollinearity_threshold = 0.7)

### 5 - Comparação e escolha dos modelos 

O Pycaret fornece uma comparação das principais métricas baseadas na acurácia, entretanto para modelos de classificação nos quais as classes da variável alvo apresentam-se muito desbalanceadas, como é o caso em nosso estudo, não devemos utilizar essa métrica.

Métricas melhores para o nosso caso seriam: Precision, Recall, F1 Score e AUC.

In [ ]:
# Comparando modelos com base no F1 + Recall
best_model = compare_models(n_select = 3, sort = 'F1')

Todos os modelos testados apresentam F1 Score bem abaixo do ideal. 

Como, para nosso objetivo, os "Falsos Negativos" são considerados mais prejudiciais que os "Falsos Positivos", analisaremos os modelos com base no F1 Score com maior Revocação (Recall), buscando os testar os modelos com níveis mais aceitáveis para o problema estudado.

In [ ]:
best = compare_models(include = ['gbc', 'ada', 'knn'])

### Criando o modelo Gradient Boosting Classifier

In [ ]:
gbc = create_model('gbc')

In [ ]:
print(gbc)

### Criando o modelo Ada Boost Classifier

In [ ]:
ada = create_model('ada')

In [ ]:
print(ada)

### Criando o modelo K Neighbors Classifier


In [ ]:
knn = create_model('knn')

In [ ]:
print(knn)

### 6 -  Matriz de confusão

In [ ]:
# Modelo Gradient Boosting Classifier
plot_model(gbc, plot = 'confusion_matrix', plot_kwargs = {'percent' : True})

In [ ]:
# Modelo Ada Boost Classifier
plot_model(ada, plot = 'confusion_matrix', plot_kwargs = {'percent' : True})

In [ ]:
# Modelo K Neighbors Classifier
plot_model(knn, plot = 'confusion_matrix', plot_kwargs = {'percent' : True})

### 7 - Análises globais de performance

In [ ]:
# Avaliação global Gradient Boosting Classifier
evaluate_model(gbc)

In [ ]:
plot_model(gbc, plot = 'auc')

In [ ]:
plot_model(gbc, plot = 'learning')

In [ ]:
plot_model(gbc, plot = 'feature_all')

In [ ]:
plot_model(gbc, plot = 'class_report')

In [ ]:
# Avaliação Ada Boost Classifier
evaluate_model(ada)

In [ ]:
plot_model(ada, plot = 'auc')

In [ ]:
plot_model(ada, plot = 'learning')

In [ ]:
plot_model(ada, plot = 'feature_all')

In [ ]:
plot_model(ada, plot = 'class_report')

In [ ]:
# Avaliação K Neighbors Classifier
evaluate_model(knn)

In [ ]:
plot_model(knn, plot = 'auc')

In [ ]:
plot_model(knn, plot = 'learning')

In [ ]:
plot_model(knn, plot = 'class_report')

### 8 - Tunning do modelo escolhido

In [ ]:
tuned_ada = tune_model(ada, optimize = 'Recall')

In [ ]:
# Comparando os modelos
# default model
print(ada)

# tuned model
print(tuned_ada)

In [ ]:
plot_model(tuned_ada, plot = 'confusion_matrix', plot_kwargs = {'percent' : True})

In [ ]:
plot_model(tuned_ada, plot = 'confusion_matrix')

In [ ]:
evaluate_model(tuned_ada)

In [ ]:
plot_model(tuned_ada, plot = 'auc')

In [ ]:
plot_model(tuned_ada, plot = 'learning')

In [ ]:
plot_model(tuned_ada, plot = 'feature_all')

In [ ]:
plot_model(tuned_ada, plot = 'class_report')

### 9 - Predição utilizando o modelo escolhido - Dados para modelagem ('data')

In [ ]:
# Predição na amostra de teste (data) - Ada Boost Classifier Tunned
predict_model(tuned_ada)

### 11 - Finalizando o modelo para implantação

In [ ]:
final_tuned_ada = finalize_model(tuned_ada)
print(final_tuned_ada)

### 12 - Predição utilizando o modelo escolhido - Dados para predição ('data_pred')

In [ ]:
unseen_predictions = predict_model(final_tuned_ada, data=data_pred)
unseen_predictions.head()

In [ ]:
unseen_predictions.info()

In [ ]:
# Verificando erros e acertos da predição
freq_acertos = unseen_predictions.groupby(['TARGET', 'Label']).size()
freq_acertos

In [ ]:
# Verificando a taxa de Recall do modelo nos dados para predição
recall = check_metric(unseen_predictions['TARGET'], unseen_predictions['Label'], metric = 'Recall')
print('A taxa de recall do modelo nos dados para predição foi de', recall)

In [ ]:
# Verificando o F1 Score do modelo nos dados para predição
F1 = check_metric(unseen_predictions['TARGET'], unseen_predictions['Label'], metric = 'F1')
print('O F1 Score do modelo nos dados para predição foi de', F1)

In [ ]:
# Verificando a taxa de Precision do modelo nos dados para predição
precision = check_metric(unseen_predictions['TARGET'], unseen_predictions['Label'], metric = 'Precision')
print('A taxa de precisão do modelo nos dados para predição foi de', precision)

### 13 - Salvando o modelo

In [ ]:
save_model(final_tuned_ada,'Final Teste ADA TUNED Model 12Abr2022')